<b> This notebook convert a mailing list (or a set of mailing lists) into a network of interaction</b>

What it does:
-it creates a network of interaction between senders and receivers of emails, on one or more mailing lists
-it generates a .gexf file that can be imported in Gephi for visualization and analysis

Parameters to set options:
-it can look in one or more mailing lists, according to how many urls are set in the ‘urls’ variable; networks are aggregated across mailing lists
-it can filter the network by date; set the variable 'date_from' and 'date_to' with a date frame consistent with the data


In [1]:
%matplotlib inline

In [2]:
from bigbang.archive import Archive
from bigbang.archive import load as load_archive
import bigbang.parse as parse
import bigbang.analysis.graph as graph
import bigbang.ingress.mailman as mailman
import bigbang.analysis.process as process
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from pprint import pprint as pp
import pytz
import os

In [3]:
# Insert a list of archive names
archives_names = ["6lo"]

cwd = os.getcwd()

archives_paths = list()
for archive_name in archives_names:
    archives_paths.append("../../archives/" + archive_name + ".csv")


archives_list = [load_archive(archive_path).data for archive_path in archives_paths]

archives = Archive(pd.concat(archives_list))

archives_data = archives.data

FileNotFoundError: [Errno 2] No such file or directory: '../../archives/6lo.csv'

Set a valid date frame for building the network. 

In [4]:
# The oldest date and more recent date for the whole mailing lists are displayed, so you WON't set an invalid time frame
print(archives_data["Date"].min())
print(archives_data["Date"].max())

2013-05-24 18:18:40+00:00
2021-02-26 08:49:30+00:00


In [5]:
# set the date frame
date_from = pd.datetime(2000, 11, 1, tzinfo=pytz.utc)
date_to = pd.datetime(2111, 12, 1, tzinfo=pytz.utc)

/home/sb/anaconda3/envs/bigbang/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  
/home/sb/anaconda3/envs/bigbang/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  This is separate from the ipykernel package so we can avoid doing imports until


Filter data according to date frame and export to .gexf file

In [6]:
def filter_by_date(df, d_from, d_to):
    return df[(df["Date"] > d_from) & (df["Date"] < d_to)]

In [7]:
# create filtered network
archives_data_filtered = filter_by_date(archives_data, date_from, date_to)
network = graph.messages_to_interaction_graph(archives_data_filtered)

In [8]:
# export the network in a format that you can open in Gephi.

# insert a file name
file_name = "architecture_discuss_for_gephi.gexf"

network = nx.write_gexf(network, cwd + file_name)